#LIBRARY

In [ ]:
%%capture
!pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split


#LOAD DATA

In [ ]:
from google.colab import drive
!gdown "1rD-_mHaqds2plYf3R20xe42NKKAEku49"

Downloading...
From: https://drive.google.com/uc?id=1rD-_mHaqds2plYf3R20xe42NKKAEku49
To: /content/databarulagi.csv
100% 48.8M/48.8M [00:01<00:00, 45.4MB/s]


In [ ]:
# Load Data
df = pd.read_csv("databarulagi.csv")

In [ ]:
df.dropna(inplace=True)

In [ ]:
df = df[df['sentimen'].isin([0, 2])]

In [ ]:
df_copy = df[['Stemmed','sentimen']]
df_copy.head()

,Stemmed,sentimen
18616,viral generasi bohong,0
18617,rusak nama anies baswedan jelek jelek nama anies,0
18618,bengkak,0
18619,kafir halang allah,0
18620,wajib presiden rakyat indonesia dukung preside...,0


In [ ]:
review = df["Stemmed"]
label = df['sentimen']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectors= TfidfVectorizer(min_df=7, ngram_range=(1,2), max_features = 5000, norm='l1')
vectors

TfidfVectorizer(max_features=5000, min_df=7, ngram_range=(1, 2), norm='l1')

In [ ]:
tfidf = vectors.fit_transform(review.astype(str))
tfidf

<37391x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 416095 stored elements in Compressed Sparse Row format>

In [ ]:
df_tfidf = tfidf.copy()

In [ ]:
label.value_counts()

2    21866
0    15525
Name: sentimen, dtype: int64

In [ ]:
x = tfidf.copy()
y = df['sentimen']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.1, random_state=42)

In [ ]:
y_train.value_counts()

2    19688
0    13963
Name: sentimen, dtype: int64

In [ ]:
y_test.value_counts()

2    2178
0    1562
Name: sentimen, dtype: int64

In [ ]:
print(x_train.shape)

(33651, 5000)


In [ ]:
from gensim.models import Word2Vec
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score

In [ ]:
from google.colab import drive
#!gdown "1UbgKJqGaBo0nUjB1KdGszVdUpZXtciOh"
#!gdown "1CUkGeGAxJIX_ObHzKSdS2epjYSsflmBJ"
#!gdown "1gFNEEIrwwlsoxi0FNKVJLkrijhzncPmA"
#berita
!gdown "1lOAO2PE1vm-CWg1dGfv8b4X12vWirLFU"
!gdown "1kyNC3SOjerhvGhFwsV3B3AdOUqOBBm62"
!gdown "11gE4o7nrT5x6FSw-cUG-XoY3FIEi52vi"

Downloading...
From: https://drive.google.com/uc?id=1lOAO2PE1vm-CWg1dGfv8b4X12vWirLFU
To: /content/w2v_berita.model
100% 8.23M/8.23M [00:00<00:00, 36.1MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1kyNC3SOjerhvGhFwsV3B3AdOUqOBBm62
From (redirected): https://drive.google.com/uc?id=1kyNC3SOjerhvGhFwsV3B3AdOUqOBBm62&confirm=t&uuid=2d874cbe-d9a2-4047-8cd9-0b989b503f3e
To: /content/w2v_berita.model.wv.vectors.npy
100% 180M/180M [00:02<00:00, 64.2MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=11gE4o7nrT5x6FSw-cUG-XoY3FIEi52vi
From (redirected): https://drive.google.com/uc?id=11gE4o7nrT5x6FSw-cUG-XoY3FIEi52vi&confirm=t&uuid=e9dc458a-bdc0-4edf-b095-ce7ceeb85897
To: /content/w2v_berita.model.syn1neg.npy
100% 180M/180M [00:03<00:00, 49.9MB/s]


In [ ]:
w2v_model = Word2Vec.load('w2v_berita.model')

In [ ]:
def feature_expansion(df,feature):
    for col in tqdm(df.columns): #loop per kolom
        try:
            sim_word = w2v_model.wv.similar_by_word(col, topn = 1) #Mencari similarity berdasarkan nilai n
        except:
            sim_word = []
        if sim_word != []: #jika similarity tidak kosong
            for term in [sim_word[i][0] for i in range(len(sim_word))]: #loop per-word yang ada di Similarity
                if term in feature:
                    #untuk semua kolom yang mempunyai nilai 0 di kolom, tetapi mempunyai nilai yang bukan 0 pada kolom term
                    #nilainya diganti dengan nilai kolom term yang mempunyai nilai bukan 0
                    df[col][(df[col]==0) & (df[term]!=0)] = df[term][(df[col]==0) & (df[term]!=0)]
    return df

In [ ]:
feature_ef = vectors.get_feature_names_out()

In [ ]:
# Membuat dataframe data test & train
df_x_train = pd.DataFrame(x_train.todense(), columns = feature_ef)
df_x_test = pd.DataFrame(x_test.todense(), columns = feature_ef)

In [ ]:
from tqdm import tqdm
dfexp_x_train= feature_expansion(df_x_train, feature_ef)

100%|██████████| 5000/5000 [01:12<00:00, 69.30it/s]


In [ ]:
dfexp_x_test = feature_expansion(df_x_test, feature_ef)

100%|██████████| 5000/5000 [01:12<00:00, 69.08it/s]


In [ ]:
x_train_model = np.expand_dims(dfexp_x_train,1)
x_test_model =np.expand_dims(dfexp_x_test,1)

In [ ]:
y_train = pd.get_dummies(y_train).values
y_test = pd.get_dummies(y_test).values
y_test

array([[0, 1],
       [1, 0],
       [0, 1],
       ...,
       [1, 0],
       [1, 0],
       [0, 1]], dtype=uint8)

In [ ]:
print(x_train_model.shape)

(33651, 1, 5000)


In [ ]:
!pip install deap

In [ ]:
import random
from deap import base, creator, tools, algorithms
import numpy as np
from keras import layers
from keras.layers import GRU, Dense, Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Define Fitness Function
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)  # Use list for mutability

# Toolbox GA
toolbox = base.Toolbox()
toolbox.register("attr_int", random.randint, 64, 128)
toolbox.register("attr_float", lambda: random.uniform(0.2, 0.6))
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_int, toolbox.attr_float), n=2)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Evaluation Function
def evaluate_ga(individual, x_train, y_train):
    gru_units = int(individual[0])
    dropout_rate = individual[1]  # Access dropout_rate directly
    model = build_gru_model(gru_units, dropout_rate)

    # Early stopping and model checkpoint
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=3),
        ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)
    ]

    model.fit(x_train, y_train, epochs=10, validation_split=0.2, callbacks=callbacks)
    _, accuracy = model.evaluate(x_train, y_train)
    return accuracy,

# Model Building Function (unchanged)
def build_gru_model(gru_units, dropout_rate):
  model = Sequential()
  model.add(layers.Input(shape=(1, 5000)))
  model.add(GRU(gru_units, activation='relu', return_sequences=True))
  model.add(Dropout(rate=dropout_rate))
  model.add(GRU(128, activation='relu', return_sequences=True))
  model.add(Dropout(rate=dropout_rate))
  model.add(GRU(64, activation='relu', return_sequences=False))
  model.add(Dropout(rate=dropout_rate))
  model.add(Dense(2, activation='softmax'))
  model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
  return model

# Configuration and Execution
x_train = x_train_model
y_train = y_train

# Register modified crossover function directly
toolbox.register("mate", tools.cxUniform, indpb=0.5)  # No need to redefine

# Register mutation function that works with lists
toolbox.register("mutate", tools.mutUniformInt, low=64, up=128, indpb=0.2)  # Adjust for integers
toolbox.register("select", tools.selTournament, tournsize=3)

toolbox.register("evaluate", evaluate_ga, x_train=x_train, y_train=y_train)
population = toolbox.population(n=5)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("max", np.max)

cxpb, mutpb, ngen = 0.5, 0.2, 5

algorithms.eaSimple(
    population, toolbox, cxpb=cxpb, mutpb=mutpb, ngen=ngen, stats=stats, halloffame=hof
)

best_ind = hof[0]
best_fitness = best_ind.fitness.values[0]

print("Best individual:", best_ind)
print("Best fitness:", best_fitness)


Epoch 1/10
842/842 [==============================] - 22s 21ms/step - loss: 0.5834 - accuracy: 0.6766 - val_loss: 0.4513 - val_accuracy: 0.7963
Epoch 2/10
842/842 [==============================] - 17s 20ms/step - loss: 0.4246 - accuracy: 0.8135 - val_loss: 0.4465 - val_accuracy: 0.7893
Epoch 3/10
842/842 [==============================] - 17s 20ms/step - loss: 0.3959 - accuracy: 0.8325 - val_loss: 0.4350 - val_accuracy: 0.8107
Epoch 4/10
842/842 [==============================] - 16s 19ms/step - loss: 0.3821 - accuracy: 0.8395 - val_loss: 0.4387 - val_accuracy: 0.8115
Epoch 5/10
842/842 [==============================] - 17s 21ms/step - loss: 0.3721 - accuracy: 0.8444 - val_loss: 0.4400 - val_accuracy: 0.8104
Epoch 6/10
842/842 [==============================] - 16s 20ms/step - loss: 0.3681 - accuracy: 0.8488 - val_loss: 0.4235 - val_accuracy: 0.8118
Epoch 7/10
842/842 [==============================] - 18s 22ms/step - loss: 0.3597 - accuracy: 0.8525 - val_loss: 0.4178 - val_accuracy: